In [4]:
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import math as math

In [30]:
def powerlaw(r, s0, b):  
    # r0 no es variable aleatoria
    r0 = 300
    return s0*(r/r0)**(-b)

def baricentro(signal,det_pos):
    bari_x = signal[0]*det_pos[0][0]
    bari_y = signal[0]*det_pos[0][1]
    sum_s = np.sum(signal)
    for i in range(1,len(signal)):
        bari_x += signal[i]*det_pos[i][0]
        bari_y += signal[i]*det_pos[i][1]
    return(bari_x/sum_s,bari_y/sum_s)
    


#estimacion inicial del core es la pos del detector mas cercano.

def funcion_costo2(theta,x_pos,y_pos,signal_sim,angle,ysigma):
        s0 = theta[0]
        b = theta[1]
        x_core = theta[2]
        y_core = theta[3]
        costo = np.zeros_like(s0)
        zenith = simZenith(-angle,angle)[0]
        azimuth = simZenith(-angle,angle)[1]        
        for (dist_x,dist_y,y1,sigma1) in zip(x_pos,y_pos,signal_sim,ysigma):
            dist = getDistances(dist_x,dist_y,x_core,y_core,zenith,azimuth)[0]
            yfit = powerlaw(dist, s0, b)
            residuo = y1 - yfit
            z = (residuo / sigma1)**2
            costo += z
        return costo
    
costo2 = lambda theta: funcion_costo2(theta,x_det,y_det,y_random,angulo,sigma) 

def in_hex(poli):
    path = mpltPath.Path(poli)
    hex = False
    random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    while hex != True:
        if path.contains_points([random_point]) == True:
            hex = True
        else:
            random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    return random_point

def getDistances(xpositions, ypositions, x0=0, y0=0, zenithRad=0, azimuthRad=0):

    dx = xpositions - x0    # detector wrt core positions
    dy = ypositions - y0
    
    groundDistances = np.sqrt(dx*dx+dy*dy)

    # Direction cosines in the plane xy
    dirx =  math.cos(azimuthRad)
    diry =  math.sin(azimuthRad)

    axisProjections = math.sin(zenithRad) * (dx*dirx+dy*diry)
    
    distances = np.sqrt( groundDistances**2 - axisProjections**2 )
    
    return (distances,axisProjections,groundDistances)

def simZenith(zenithMinDeg, zenithMaxDeg) :

    zenithMinRad = math.radians(zenithMinDeg)
    sin2MinZenith = math.sin(zenithMinRad)**2

    zenithMaxRad = math.radians(zenithMaxDeg)
    sin2MaxZenith = math.sin(zenithMaxRad)**2

    rng = np.random.default_rng()
    u = rng.random() 
    
    sin2Zenith = sin2MinZenith + u * (sin2MaxZenith-sin2MinZenith)
    sinZenith = math.sqrt(sin2Zenith) 

    zenith = math.asin(sinZenith)
    azimut = np.random.uniform(0,2*np.pi)
    
    return (zenith,azimut)

poligono = [[-216.5,-375],[216.5,-375],[433,0],[216.5,375],[-216.5,375],[-433,0]]
detectores_pos = [[-216.5,-375],[216.5,-375],[433,0],[216.5,375],[-216.5,375],[-433,0],[0,0]]
rectangulo = [[-433,-375],[433,-433],[433,375],[-433,375]]
pointsx,pointsy = [-216.5,216.5,433,216.5,-216.5,-433.5,0] , [-375,-375,0,375,375,0,0]
random_point = in_hex(poligono)
centro = (0,0)
x_det = np.zeros(len(detectores_pos))
y_det = np.zeros(len(detectores_pos))
for i in range(len(detectores_pos)):
    x_det[i] = detectores_pos[i][0]
    y_det[i] = detectores_pos[i][1]

In [48]:
#Defino parametros de LDF 
r0 = 300 #metros 
s0 = 35 #VEM
angulo = 40
b = 2.1    
#Genero el punto random
random_point = in_hex(poligono)
#calculo las distancias entre el evento y los 7 detectores.
distancias = getDistances(np.asarray(pointsx),np.asarray(pointsy),random_point[0],random_point[1],simZenith(-angulo,angulo)[0],simZenith(-angulo,angulo)[1])
r = np.asarray(distancias[0])

signal = powerlaw(r,s0,b)
sigma_signal = np.sqrt(signal)
#cortar detectores con señal menores a dos 
#Hago el ajuste y defino la semilla inicial
y_random = np.random.normal(signal,sigma_signal,size = len(signal)) #signal
y_random = y_random[y_random > 2]
sigma = np.sqrt(y_random)


core_est_x = baricentro(y_random,detectores_pos)[0] 
core_est_y = baricentro(y_random,detectores_pos)[1] 


In [60]:
res = minimize(costo2, x0=(30,1,core_est_x,core_est_y))
print('Minimize results')
print("Parametros ajustados: ", res.x)
cova = 2*res.hess_inv
par_error = np.sqrt(np.diagonal(cova))
print("Errores de los parámetros", par_error)
print(cova)

Minimize results
Parametros ajustados:  [  30.00001665    1.00001418  176.70398056 -168.08607329]
Errores de los parámetros [2.97870301 1.96738214 1.47244003 0.15281589]
[[ 8.87267165  5.5087378  -4.32505699 -0.42295578]
 [ 5.5087378   3.87059249 -2.52116999 -0.30050744]
 [-4.32505699 -2.52116999  2.16807964  0.19236089]
 [-0.42295578 -0.30050744  0.19236089  0.0233527 ]]


/Users/gunitettamanti/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:1058: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))


In [61]:
random_point

(194.37733628308342, -176.73703036075568)